In [1]:
import random
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
import numpy as np

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
import math
import scipy.optimize as optim


In [4]:
def do_hist(x, bins=10, width = 550, height = 550, title = None):
  hist, edges = np.histogram(x, bins = bins, density=True)
  p_hist = figure(width = width, height = height, title = title)
  p_hist.quad(top = hist, bottom=0, left=edges[:-1], right=edges[1:], alpha = .4)
  return p_hist

In [5]:
X = [random.gauss(mu = 0, sigma = 1) for x in range(100)]
X2 = [x**2 for x in X]
p = do_hist(X2)
show(p)

In [6]:
X =[random.gauss(mu = 0, sigma = 1) for x in range(100)]
X3 = [random.gauss(mu = -1000, sigma = 1) for x in range(100)]
zz = zip(X, X3)
dff = [(x[0] - x[1]) **2  for x in zz]
p  =do_hist(dff)
show(p)


In [7]:
X = [random.gauss(mu = 0, sigma = 1) for x in range(100)]
X2 = [random.gauss(mu = 1, sigma = 1) for x in range(100)]
#X is data, X2 is some type of fit
#calculate SSR from mean
m1 = np.mean(X)
ssr_mean= [(x-m1) **2 for x in X]
ssr_mean
zz = zip(X,X2)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
ssr_fit

def resample_two_samples(sample1, sample2, num_iterations = 100):
    both = sample1 + sample2
    sum1 = []
    sum2 = []
    for i in range(num_iterations):
        random.shuffle(both) 
        new_1 = both[0:len(sample1)] 
        new_2 = both[len(sample1):] 
        sum1.append(np.mean(new_1)) 
        sum2.append(np.mean(new_2)) 
    return sum1, sum2

def combine_resamples(sample1, sample2, resample1, resample2):
    diff1 = [np.mean(sample1 )- x for x in resample1]
    diff2 = [np.mean(sample2)- x for x in resample2]
    both = np.array(diff1) - np.array(diff2)
    return both
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
sum(ssr_mean)
len([x for x in l if x > 1])


0

In [9]:
def exp_func(x, initial, ratio):
    return initial * np.power(ratio, x - 1)
X = list(range(25))
initial = 1
ratio = 2
X = list(range(1, 25))
y = [exp_func(x, initial, ratio ) + random.gauss(mu = 0, sigma = 1) for x in X]

popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(y) )
initial_hat = popt[0]
ratio_hat = popt[1]
ssr_mean =[(x-np.mean(y)) **2 for x in y]
y_fit = [exp_func(x = x, initial= initial_hat, ratio = ratio_hat) for x in X]
zz = zip(y, y_fit)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
len([x for x in l if x > 0])

100

In [ ]:

X = list(range(1, 25))
y = [ random.gauss(mu = 10, sigma = 1) for x in X]
popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(y) )
initial_hat = popt[0]
ratio_hat = popt[1]
ssr_mean =[(x-np.mean(y)) **1 for x in y]
y_fit = [exp_func(x = x, initial= initial_hat, ratio = ratio_hat) for x in X]
zz = zip(y, y_fit)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
